In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from ensemble_classes import RfRegressor, Ensemble
import matplotlib.pyplot as plt
from time import time

In [ ]:
def fix_drug_ordering(x, y):
    new_y = []
    for c in x.columns:
        new_y.append(y.loc[c])
    
    return new_y

In [ ]:
train_response_df = pd.read_table('Project/DREAM_train/DREAM7_DrugSensitivity1_Drug_Response_Training.txt', index_col=0)
#print(train_response_df.shape) # 31 drugs, 35 cell lines
#train_response_df.head() # each row is a cell-line, each col is a drug

test_response_df = pd.read_table('Project/DREAM_test/DREAM7_DrugSensitivity1_test_data.txt', index_col=0)
#print(test_response_df.shape) # 31 drugs, 18 cell lines 
#test_response_df.head() # each row is a cell-line, each col is a drug

rppa = pd.read_csv('Project/DREAM_train/DREAM7_DrugSensitivity1_RPPA.txt', sep='\t')

rppa.head()

rppa_valid = rppa.query("FullyValidated == 'Yes'")
rppa_valid = rppa_valid.iloc[:, 2:]

rppa_valid.head()


train_cols = [x for x in rppa_valid.columns if x in list(train_response_df.index)]
test_cols = [x for x in rppa_valid.columns if x in list(test_response_df.index)]

for t in test_cols:
    train_cols.append(t)
    
cols = train_cols

scaled_X = pd.DataFrame(data=StandardScaler().fit_transform(rppa_valid), columns=rppa_valid.columns)


rppa = scaled_X[cols].T

rppa.head()

# train_Y = train_response_df.loc[train_rows]
# print(train_Y.shape)
# test_Y = test_response_df.loc[test_rows]
# print(test_Y.shape)

# print(train_Y.isna().sum())
# print(train_Y['Drug20'])


# y_ordered = fix_drug_ordering(train_X, train_Y['Drug20'])


# train_X = train_X.T
# features = ['prot%i' % i for i in range(len(train_X.columns))]
# train_X.columns = features
# train_X['y'] = y_ordered

# U = train_X
# L = pd.DataFrame(columns=U.columns)
# rf_reg = RfRegressor(L, U, features)

# for i in U.index[:10]:
#     rf_reg.update_model(i)
    
# mse = rf_reg.get_mse_on_L()


# for c in U.index[10:]:
#     print(rf_reg.get_model_uncertainty(c))

,0,1,2,3,4,5,6,7,8,9,...,56,57,58,59,60,61,62,63,64,65
AU565,-1.395512,-0.443920,0.460631,0.091295,-0.507581,-0.463707,-2.533863,0.056588,-2.256236,-0.696438,...,0.484206,0.501783,-0.366054,-0.136749,-0.300745,-0.898594,3.286737,5.184789,0.287299,0.819693
BT20,-0.386343,-0.150141,-2.000338,1.773835,1.869955,-1.274331,1.044535,-0.399582,1.320288,1.027531,...,-0.644660,-1.976989,-0.027221,-0.124317,0.300623,-2.243376,-1.026624,-2.370816,-0.841966,-0.290435
BT474,-0.527699,-0.605438,0.323833,-0.037625,0.809474,0.377850,0.193741,0.628568,-3.191386,-0.788083,...,-0.599308,1.340810,-0.082292,0.187866,1.027141,0.095433,2.925587,4.017672,-0.309056,0.312394
BT483,1.349909,1.340035,0.154638,-0.723010,-1.817873,-0.601932,-0.495536,-1.356085,-2.769201,-0.850860,...,-0.640211,0.129240,0.725335,-0.305176,2.391737,0.194567,-0.006449,-0.144240,-0.216325,0.278167
BT549,1.205447,0.918449,-0.556271,1.263726,1.583743,-0.184698,0.777911,0.981800,3.021390,0.898606,...,0.161882,-0.050295,-0.615340,-0.101458,-0.940172,1.675021,-1.835587,-2.067737,-0.395701,0.085356


In [ ]:
# test_e = Ensemble([train_X], init_size=2)

In [ ]:
# test_e.iterate()

In [ ]:
gene_exp = pd.read_csv('Project/cell_line_data/gene_exp_df.csv', sep='\t', index_col=0)
ge_columns = ['g%s' % i for i in gene_exp.columns[:-1]]
ge_columns.append('y')
gene_exp.columns = ge_columns

meth = pd.read_csv('Project/cell_line_data/methylation_df.csv', sep='\t', index_col=0)
meth_cols = ['p%s' % i for i in meth.columns[:-1]]
meth_cols.append('y')
meth.columns = meth_cols

rna_seq = pd.read_csv('Project/cell_line_data/RNAseq_df.csv', index_col=0)

snp = pd.read_csv('Project/cell_line_data/SNP6_df.csv', sep='\t', index_col=0)
snp_cols = ['s%s' % i for i in snp.columns[:-1]]
snp_cols.append('y')
snp.columns = snp_cols

y_true = gene_exp['y']

y_dict = {}
for y, i in zip(y_true.values, y_true.index):
    y_dict[i] = y

In [ ]:
cells = []
with open('Project/cell_line_data/intersect_cells.txt', 'r') as f:
    for line in f:
        c = line.strip()
        if c != 'ZR751' and c != 'SUM52PE':
            cells.append(line.strip())
        
print(cells)

rppa = rppa.loc[cells, :]
rppa_y = []

for i in rppa.index:
    rppa_y.append(y_dict[i])

    
rppa['y'] = rppa_y  
rppa = rppa.sort_index(axis=0)

['SKBR3', 'HS578T', 'MCF7', 'MCF12A', 'MDAMB453', 'BT549', 'HCC1143', 'HCC38', 'HCC1954', '600MPE', 'HCC1937', 'HCC3153', 'MDAMB361', 'LY2', 'BT474', 'SUM149PT', 'BT483', 'HCC1428', 'MDAMB231', 'SUM225CWN', 'AU565', 'HCC202', 'SUM159PT', 'UACC812', 'HCC70', 'CAMA1', 'T47D', 'MCF10A']


In [ ]:
gene_exp = gene_exp.loc[cells, :].sort_index(axis=0)
meth = meth.loc[cells, :].sort_index(axis=0)
rna_seq = rna_seq.loc[cells, :].sort_index(axis=0)
snp = snp.loc[cells, :].sort_index(axis=0)
snp = snp.dropna(axis=1)

In [ ]:
def get_mse(inds, y_true, pred):
    yt = y_true.loc[inds].values
    tot = 0
    
    for y, yh in zip(yt, pred):
        tot += (y-yh)**2
        
    return tot/len(yt)

In [ ]:
# #Uncertainty
# train_error = []
# test_error = []

# for i in range(10):
#     print('Round %i' % i)
#     ensemble = Ensemble([gene_exp, meth, rna_seq, snp, rppa], init_size=3, seed=i)
#     y_true = gene_exp['y'].copy()

#     train_mse = []
#     test_mse = []
#     done = False

#     o_inds = ensemble.init_inds
#     u_inds = list(y_true.index)

#     for o in o_inds:
#         u_inds.remove(o)

#     while not done:
#         t0 = time()
#         o_pred, i, done, u_shape = ensemble.iterate()

#         o_inds = np.append(o_inds, i)
#         u_inds.remove(i)

#         mse = get_mse(o_inds, y_true, o_pred)
#         train_mse.append(mse)

#         if not done:
#             t_pred, u_ind = ensemble.predict_unobserved()
#             mse = get_mse(u_inds, y_true, t_pred)
#             test_mse.append(mse)
            
            
#     train_error.append(train_mse)
#     test_error.append(test_mse)


In [ ]:
# plt.figure()
# plt.title('Train MSE vs Round #')
# plt.plot(range(len(train_mse)), train_mse)
# plt.show()
# plt.close()

In [ ]:
# print(test_mse)

# plt.figure()
# plt.title('Test MSE vs Round #')
# plt.plot(range(len(test_mse)), test_mse)
# plt.show()
# plt.close()

In [ ]:
arr = np.asarray([[1,2,3], [4,5,6]])

In [ ]:
# np.savetxt('Train_error_10_rand.csv', train_error, delimiter=',')
# np.savetxt('Test_error_10_rand.csv', test_error, delimiter=',')

In [ ]:
test_rand_error = []
train_rand_error = []

for i in range(2):
    print('Round %i' % i)
    ensemble = Ensemble([gene_exp, meth, rna_seq, snp, rppa], init_size=3, seed=i)
    y_true = gene_exp['y'].copy()

    train_mse = []
    test_mse = []
    done = False

    o_inds = ensemble.init_inds
    u_inds = list(y_true.index)

    for o in o_inds:
        u_inds.remove(o)

    while not done:
        t0 = time()
        o_pred, i, done, u_shape = ensemble.iterate(mode='rand')

        o_inds = np.append(o_inds, i)
        u_inds.remove(i)

        mse = get_mse(o_inds, y_true, o_pred)
        train_mse.append(mse)

        if not done:
            t_pred, u_ind = ensemble.predict_unobserved()
            mse = get_mse(u_inds, y_true, t_pred)
            test_mse.append(mse)
            
            
    train_rand_error.append(train_mse)
    test_rand_error.append(test_mse)

Round 0


In [ ]:
uc_test = np.loadtxt('Test_error_10_rand.csv', delimiter=',')

In [ ]:
fig, ax = plt.subplots() 

uc_avg = np.mean(uc_test, axis=0)
uc_std = np.std(uc_test, axis=0)
uc_ub = uc_avg + uc_std
uc_lb = uc_avg - uc_std

p = ax.plot(range(len(uc_avg), uc_avg, label='uncertainty')
ax.fill_between(range(nsteps+1), uc_lb, uc_ub, color=p[0].get_color(), alpha=0.15)
            
rand_avg = np.mean(test_rand_error, axis=0)
rand_std = np.std(test_rand_error, axis=0)
rand_ub = rand_avg + rand_std
rand_lb = rand_avg - rand_std

p = ax.plot(range(len(rand_avg), rand_avg, label='random')
ax.fill_between(range(len(rand_avg)), rand_lb, rand_ub, color=p[0].get_color(), alpha=0.15)
ax.set_title("Test MSE as a function of round number")
ax.set_xlabel("Round Number")
ax.set_ylabel("MSE")                   
            
ax.legend()
            
plt.show()